# Double Pendulum Using Lagrangian Mechanics

Uses numerical DiffEq solution to find the behavior of a double pendulum using Lagrange's equations

Last Edited: 4/29/22 By: Matthew Freudenrich


The double pendulum being solved is characterized by two generalized coordinates, $\phi$ and $\theta$. These define the angle of the first and second arms respectively to the vetical.


From section 11.4 in the textbook, we know the potential and kinetic energy of the double pendulum

$U =(m_1 + m_2)gL_1 - cos(\phi)(m_1+m_2)gL_1 + m_2gL_2 - m_2gL_2cos(\theta)$

$T = \frac{1}{2}(m_1+m_2)L_1^2\dot{\phi}^2 +\frac{1}{2}m_2L_2^2\dot{\theta}^2 + m_2L_1L_2\dot{\phi}\dot{\theta}cos(\phi-\theta)$


The Lagrangian can now be defined 

$\mathcal{L} = T - U = \frac{1}{2}(m_1+m_2)L_1^2\dot{\phi}^2 +\frac{1}{2}m_2L_2^2\dot{\theta}^2 + m_2L_1L_2\dot{\phi}\dot{\theta}cos(\phi-\theta) -(m_1 + m_2)gL_1 +cos(\phi)(m_1+m_2)gL_1 -m_2gL_2 + m_2gL_2cos(\theta)$

Solving the lagrange equation for the generalized variable $\phi$ gives 

$\frac{\partial \mathcal{L} }{\partial \phi} = \frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot{\phi}}$

$-m_2L_1L_2\dot{\phi}\dot{\theta}sin(\phi-\theta)-(m_1+m_2)gL_1sin(\phi) = (m_1+m_2)L_1^2\ddot{\phi} + m_2L_1L_2(\ddot{\theta}cos(\theta-\phi)-\dot{\theta}(\dot{\phi}-\dot{\theta})sin(\phi-\theta))$

Solving the lagrange equation for the generalized variable $\theta$ gives

$\frac{\partial \mathcal{L} }{\partial \theta} = \frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot{\theta}}$

$m_2L_1L_2\dot{\phi}\dot{\theta}sin(\phi-\theta)-m_2gL_2sin(\theta) = m_2L_2^2\ddot{\theta}+m_2L_1L_2(\ddot{\phi}cos(\phi-\theta)-\dot{\phi}(\dot{\phi}-\dot{\theta})sin(\phi-\theta))$

This pair of coupled equations gives expressions for $\ddot{\phi}$ and $\ddot{\theta}$

$\ddot{\phi} = \frac{2\dot{\theta}^2L_2m_2sin(\theta-\phi) + \dot{\phi}^2L_1m_2sin(2(\theta-\phi))+g(m_2sin(2\theta-\phi)-2m_1sin(\phi)-m_2sin(\phi))}{2L_1(m_1+m_2-m_2cos^2(\theta-\phi))}$

$\ddot{\theta} = \frac{-(sin(\theta-\phi))(\dot{\theta}^2L_2m_2cos(\theta-\phi)+(m_1+m_2)(\dot{\phi}^2L_1+gCos(\phi)))}{L_2(m_1+m_2-m_2cos^2(\theta-\phi))}$


The following code defines the class used to solved the pendulum

In [ ]:
%matplotlib inline
import numpy as np
from scipy.integrate import odeint, solve_ivp

import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

class LagrangianPendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a simple pendulum (no driving or damping).
     
    Parameters
    ----------
    L : float
        length of the simple pendulum
    g : float
        gravitational acceleration at the earth's surface
    omega_0 : float
        natural frequency of the pendulum (\sqrt{g/l} where l is the 
        pendulum length) 
    mass : float
        mass of pendulum

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1., L2=1., m1=1.,m2=1., g=1.):
        self.L1 = L1
        self.L2 = L2
        self.g = g
        self.m1 = m1
        self.m2 = m2
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2 dtheta/dt d^2theta/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 4-component vector with y[0] = phi(t) and y[1] = dphi/dt
            with y[2]=theta(t) and y[3]=dtheta/dt
            
        Returns
        -------
        
        """
        return [y[1],(2.*y[3]**2*L2*m2*np.sin(y[2]-y[0])+y[1]**2*L1*m2*np.sin(2*(y[2]-y[0])) +g*(m2*np.sin(2.*y[2]-y[0]) - 2*m1*np.sin(y[0]) -m2*np.sin(y[0])) )/(2.*L1*(m1+m2-m2*(np.cos(y[2]-y[0]))**2)), y[3], (-1.)*(np.sin(y[2]-y[0])*(y[3]**2*L2*m2*np.cos(y[2]-y[0])+(m1+m2)*(y[1]**2*L1 + g*np.cos(y[0]))))/(L2*(m1+m2-m2*(np.cos(y[2]-y[0]))**2))]
    
    def solve_ode(self, t_pts, phi_0, phi_dot_0, theta_0, theta_dot_0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi_0, phi_dot_0, theta_0, theta_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi, phi_dot, theta, theta_dot = solution.y
        #print(phi, phi_dot, theta, theta_dot) #Prints only 8 values? Why are there not more values?
        return phi, phi_dot, theta, theta_dot

The following code defines the plotting functions

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

We can now plot the solution to the double pendulum. 

In [ ]:
# Labels for individual plot axes
phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt(t)$')
theta_vs_time_labels = (r'$t$', r'$\theta(t)$')
theta_dot_vs_time_labels = (r'$t$', r'$d\theta/dt(t)$')
phi_state_space_labels = (r'$\phi$', r'$d\phi/dt$')
theta_state_space_labels = (r'$\theta$', r'$d\theta/dt$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  


#Pendulum parameters
L1 = 1.
L2 = 1.
m1 = 1.
m2 = 1.
g = 1.

#Initiate a pendulum

p1 = LagrangianPendulum(L1 = L1, L2 = L2, m1 = m1,m2=m2, g = g)

#Initial conditions
phi_0 = 0
phi_dot_0 = 0.
theta_0 = 0.1
theta_dot_0 = 0.
phi, phi_dot, theta, theta_dot = p1.solve_ode(t_pts, phi_0, phi_dot_0, theta_0, theta_dot_0)


In [ ]:
# start the plot!
fig = plt.figure(figsize=(15,5))
overall_title = 'Double Pendulum Using Lagrangian Mechanics  ' + '\n' + \
                rf' $L1 = {L1:.2f},$' + \
                rf' $L2= {L2:.2f},$' + \
                rf' $\theta_0 = {theta_0:.2f},$' + \
                rf' $\dot\theta_0 = {theta_dot_0:.2f},$' + \
                rf' $g = {g:.2f},$' + \
                rf'  $\phi_0 = {phi_0:.2f},$' + \
                rf' $\dot\phi_0 = {phi_dot_0:.2f}$' + \
                '\n'  
fig.suptitle(overall_title, va='baseline')

# first plot: phi vs time
ax_a = fig.add_subplot(1,6,1)                  

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], phi[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=None, 
            title='$\phi(t)$', 
            ax=ax_a)    
                          
# second plot: phi dot vs time 
ax_b = fig.add_subplot(1,6,2)                  

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], phi_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='blue',
            label=None, 
            title='$\dot\phi(t)$', 
            ax=ax_b)    

# third plot: phi state space plot from t=30 to t=50   

ax_c = fig.add_subplot(1,6,3)                  

start, stop = start_stop_indices(t_pts, 0., 50.)    
plot_y_vs_x(phi[start : stop], phi_dot[start : stop], 
            axis_labels=phi_state_space_labels, 
            color='blue',
            label=None, 
            title=rf'$30 \leq t \leq 50$', 
            ax=ax_c)    

#Fourth plot: theta vs time
ax_d = fig.add_subplot(1,6,4)                  

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], theta[start : stop], 
            axis_labels=theta_vs_time_labels, 
            color='blue',
            label=None, 
            title='$\theta(t)$', 
            ax=ax_d)    
                      
#Fifth plot: theta dot vs time    
ax_e = fig.add_subplot(1,6,5)                  

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], theta_dot[start : stop], 
            axis_labels=theta_dot_vs_time_labels, 
            color='blue',
            label=None, 
            title='$\dot\theta(t)$', 
            ax=ax_e)   

#Sixth plot: theta state space:

ax_f = fig.add_subplot(1,6,6)                  

start, stop = start_stop_indices(t_pts, 0., 50.)    
plot_y_vs_x(theta[start : stop], theta_dot[start : stop], 
            axis_labels=theta_state_space_labels, 
            color='blue',
            label=None, 
            title=rf'$30 \leq t \leq 50$', 
            ax=ax_f)    

fig.tight_layout()

We can now check for whether the system behaves chaotically. In a chaotic system, two initial conditions seperated by a small amount $\Delta \phi$ will change over time according to: $\Delta \phi \approx e^{\lambda t}$ with $\lambda > 0$ For a doulbe pendulum, large angles from the verticle will create a chaotic behavior.

A nonchaotic system with two initial conditions seperated by a small amount $\Delta \phi$ will change over time according to $\Delta \phi \approx e^{-\lambda t}$ with $\lambda > 0$ with $\Delta \phi$ trending to zero a small constant. For a double pendulum, small angles around $\phi = \theta = 0$ will behave nonchaotically. 

Two sets of initial conditions were tested. The first should behave chaotically, while the second should behave nonchaotically. 

In [ ]:
# one plot with multiple initial conditions: should be chaotic
phi_0_1 = np.pi/2
phi_dot_0 = 0.0
theta_0_1 = -np.pi/2
theta_dot_0 = 0.0
phi_1, phi_dot_1, theta_1, theta_dot_1 = p1.solve_ode(t_pts, phi_0_1, phi_dot_0, theta_0_1, theta_dot_0)


phi_0_2 = np.pi/2
phi_dot_0 = 0.0
theta_0_2 = -np.pi/2+0.001
theta_dot_0 = 0.0
phi_2, phi_dot_2, theta_2, theta_dot_2 = p1.solve_ode(t_pts, phi_0_2, phi_dot_0, theta_0_2, theta_dot_0)


# Calculate the absolute value of \theta_2 - \theta_1
Delta_theta = np.fabs(theta_2 - theta_1) + 1e-12


# start the plot!
fig = plt.figure(figsize=(8,8))
overall_title = 'Chaotic Regieme of double pendulum ' + '\n' + \
                rf' $L1 = {L1:.2f},$' + \
                rf' $L2= {L2:.2f},$' + \
                rf' $g = {g:.2f},$' + \
                rf'$m1 = {m1:.2f},$' + \
                rf'$m2 = {m2:.2f},$' + \
                '\n'    # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# two plot: plot from t=0 to t=8 and another from t=0 to t=100 

ax_a = fig.add_subplot(2,1,1)                  

start, stop = start_stop_indices(t_pts, 0., 8.)
ax_a.semilogy(t_pts[start : stop], Delta_theta[start : stop], 
            color='blue', label=None) 
ax_a.set_xlabel('t')
ax_a.set_ylabel(r'$|\Delta\theta|$')

ax_b = fig.add_subplot(2,1,2)                  

start, stop = start_stop_indices(t_pts, 0., 100.)
plot_y_vs_x(t_pts[start : stop], Delta_theta[start : stop], 
            color='blue', label=None, semilogy=True)    
ax_b.set_xlabel('t')
ax_b.set_ylabel(r'$|\Delta\theta|$')
                              
fig.tight_layout()
# always bbox_inches='tight' for best results.  Further adjustments also.

In [ ]:
# one plot with multiple initial conditions: should be non-chaotic
phi_0_1 = 0.0
phi_dot_0 = 0.0
theta_0_1 = 0.0
theta_dot_0 = 0.0
phi_1, phi_dot_1, theta_1, theta_dot_1 = p1.solve_ode(t_pts, phi_0_1, phi_dot_0, theta_0_1, theta_dot_0)


phi_0_2 = 0.0
phi_dot_0 = 0.0
theta_0_2 = 0.0001
theta_dot_0 = 0.0
phi_2, phi_dot_2, theta_2, theta_dot_2 = p1.solve_ode(t_pts, phi_0_2, phi_dot_0, theta_0_2, theta_dot_0)


# Calculate the absolute value of \phi_2 - \phi_1
Delta_phi = np.fabs(phi_2 - phi_1) + 1e-12


# start the plot!
fig = plt.figure(figsize=(8,8))
overall_title = 'Nonchaotic Regieme of double pendulum ' + '\n' +\
                rf' $L1 = {L1:.2f},$' + \
                rf' $L2= {L2:.2f},$' + \
                rf' $g = {g:.2f},$' + \
                rf'$m1 = {m1:.2f},$' + \
                rf'$m2 = {m2:.2f},$' + \
                '\n'    # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# two plot: plot from t=0 to t=8 and another from t=0 to t=100 
ax_a = fig.add_subplot(2,1,1)                  

start, stop = start_stop_indices(t_pts, 0., 8.)
ax_a.semilogy(t_pts[start : stop], Delta_phi[start : stop], 
            color='blue', label=None) 
ax_a.set_xlabel('t')
ax_a.set_ylabel(r'$|\Delta\phi|$')

ax_b = fig.add_subplot(2,1,2)                  

start, stop = start_stop_indices(t_pts, 0., 100.)
plot_y_vs_x(t_pts[start : stop], Delta_phi[start : stop], 
            color='blue', label=None, semilogy=True)    
ax_b.set_xlabel('t')
ax_b.set_ylabel(r'$|\Delta\phi|$')
                              
fig.tight_layout()
# always bbox_inches='tight' for best results.  Further adjustments also.